# Create 4-deg (latitude) x 5-deg (longitude) monthly average gridded dataset for SIF and XCO2
---

In [1]:
import sys
sys.path.insert(0, "../src")

import pandas as pd 
import xarray

import data_utils

In [2]:
DS_sif = xarray.open_dataset("../data/exp_pro/OCO2_Lite_SIF740.nc")
DS_xco2 = xarray.open_dataset("../data/exp_pro/OCO2_Lite_XCO2.nc")

In [3]:
DS_xco2.lon.min()

<xarray.DataArray 'lon' ()>
array(-180., dtype=float32)

In [4]:
df_sif_monthly = (
    data_utils.regrid(DS_sif, lon_res=5, lat_res=4, lat_lwr=-88, lat_upr=88) # crop poles to center on equator
    .groupby(["lon", "lat"])
    .resample("1MS", on="time")
    .mean()
    .drop(columns=["lon", "lat"])
    .reset_index()
)
df_xco2_monthly = (
    data_utils.regrid(DS_xco2, lon_res=5, lat_res=4, lat_lwr=-88, lat_upr=88) # crop poles to center on equator
    .groupby(["lon", "lat"])
    .resample("1MS", on="time")
    .mean()
    .drop(columns=["lon", "lat"])
    .reset_index()
)

# Merge dataframes
DS_grid = (
    pd.merge(df_sif_monthly, df_xco2_monthly, on=["lon", "lat", "time"], how="outer")
    .set_index(["lon", "lat", "time"])
    .to_xarray()
)
DS_grid

<xarray.Dataset>
Dimensions:   (lat: 43, lon: 72, time: 75)
Coordinates:
  * lon       (lon) float64 -177.5 -172.5 -167.5 -162.5 ... 167.5 172.5 177.5
  * lat       (lat) float64 -86.0 -82.0 -78.0 -74.0 ... 70.0 74.0 78.0 82.0
  * time      (time) datetime64[ns] 2014-09-01 2014-10-01 ... 2020-11-01
Data variables:
    sif       (lon, lat, time) float32 nan nan 0.19341354 ... nan nan nan
    sif_var   (lon, lat, time) float32 nan nan 0.30232257 ... nan nan nan
    xco2      (lon, lat, time) float32 nan nan nan nan nan ... nan nan nan nan
    xco2_var  (lon, lat, time) float32 nan nan nan nan nan ... nan nan nan nan

In [5]:
DS_grid.to_netcdf("../data/exp_pro/OCO2_4x5deg_monthly.nc", format="NETCDF4")
DS_grid.close()